In [2]:
from dataclasses import dataclass
from pathlib import Path
import os

import yfinance as yf
import requests
import pandas as pd

In [4]:
%pwd

'c:\\Users\\Admin\\PhD Projects\\balance_continuous\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\Admin\\PhD Projects\\balance_continuous'

In [ ]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    local_data_file: Path

In [3]:
import pandas as pd
import requests
from io import StringIO

# Add proper headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

link = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks"

# Make the request with headers
response = requests.get(link, headers=headers)
response.raise_for_status()  # Raises an exception for bad status codes

# Parse the HTML content
df = pd.read_html(StringIO(response.text), header=0)[0]
df.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [9]:
df['Symbol']

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
498     XYL
499     YUM
500    ZBRA
501     ZBH
502     ZTS
Name: Symbol, Length: 503, dtype: object

In [12]:
df['Symbol'].to_csv('SandP500Ticker.csv', header=None, index=0)

In [ ]:
from sb_project.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
            self, data_filepath = DATA_FILE_PATH
    ):
        self.data = read_yaml(data_filepath) 

In [ ]:
def download_data(ticker:str, start_date=None, interval:str='1d'):
    """
    Download stock data from Yahoo Finance.
    """
    session = requests.Session(impersonate="chrome")
    if start_date is None:
        start_date = '2000-01-01'
    else:
        start_date = pd.to_datetime(start_date)
    data = yf.download(ticker, start=start_date, interval=interval, auto_adjust=False, session=session)
    data.columns = data.columns.droplevel(1)  # Drop the first level of the column index
    # data.reset_index(inplace=True)  # Reset the index to make 'Date' a column
    return data